In [1]:
from __future__ import print_function
import iris
import controlstage
import controlfli
reload(controlfli)
import numpy as np
import matplotlib.pylab as plt
#%matplotlib wx
import sys
sys.path.append("C:\\Users\\PSM-0\\Desktop\\PYTHON\\GIT")
import seaborn as sns
sns.set_style("white")

C:\Users\PSM-0\Desktop\PYTHON\HPF_FRD


C:\Users\PSM-0\Anaconda2\lib\site-packages\fli-dev-py2.7.egg\FLI\lib.py:403: Warning: function 'FLISetTDI' not found
  warnings.warn(Warning(err))


In [2]:
print(1)

1


In [3]:
import numpy as np
import pandas as pd
from PyAPT import APTMotor
import time
import os

class ControlStage(object):
    
    def __init__(self):
        self.M = APTMotor(83829712,HWTYPE=31,verbose=True)
        self.df = pd.read_csv('sequence.csv')
        self.x = self.df.x_out_fiber_dist.values
        
    def go_home(self):
        self.M.go_home()
    
    def go_to_32(self):
        # Move absolute
        self.M.mAbs(32.)
        
    def go_to(self,value):
        self.M.mAbs(value)
    
    def exit(self):
        self.M.cleanUpAPT()

    def run_sequence(self):
        print("Running sequence")
        for i, x in enumerate(self.x):
            self.go_to(x)
            print("##########################################")
            print(i,"Now at",x,"mm")
            input_val = raw_input("Press Enter to continue. Press q to stop")
            if input_val == "q":
                break
                
class FRDSequencer(object):
    """
    An object to run an automated FRD sequence
    """
    CSV_SETUP_SUBDIR = "..\\ANALYSIS\\CSV_SETUP\\"
    
    def __init__(self,csv_setup_file='setup_f01.csv',
                      savefolder="C:\\Users\\PSM-0\\Desktop\\HPF_FRD\\20170801_testing_sequence\\f01"):
        print("Connecting to devices...",end="")
        self.S = ControlStage()                  # Stage
        print("Stage...",end="")
        self.C = controlfli.ControlFLI(savefolder=savefolder)         # Camera
        print("Camera...",end="")
        self.I = iris.ControlIris(device='COM8') # Iris
        print("Stage...",end="")
        self.I.connect()
        print("Done")
        print("Reading sequence file...",end="")
        self.df = pd.read_csv(savefolder+os.sep+self.CSV_SETUP_SUBDIR+csv_setup_file)
        self.x = self.df['x_out_fiber_dist'].values
        self.exptime = self.df["exptime"].values
        self.filenames = self.df['filename'].values
        self.iris_diam = self.df['iris_diam_perc'].values.astype(int)
        print("Done")
        self.savefolder = savefolder
        
        
    def init(self):
        self.S.go_home()
        self.I.set_percent(18)
        print("Initialized")
        
    def run_sequence(self,overwrite=False,exptime_test=0.2):
        print("##########################################")
        print("##########################################")
        print("Running sequence")
        
        print("Savefolder:",self.savefolder)
        print("##########################################")
        #self.fig, self.ax = plt.subplots()
        self.fig, self.ax = plt.subplots(ncols=2,nrows=1,figsize=(18,10))
        
        print("Setting the percent as 18 in the beginning")
        #self.I.set_percent(18)
        
        for i in range(len(self.df)):
            print("##########################################")
            print("PERCENT=:",self.iris_diam[i])
            self.I.set_percent(self.iris_diam[i])
            self.S.go_to(self.x[i])
            print(i,"STAGE AT=",self.x[i],"mm")
            time.sleep(0.1)
            [ax.clear() for ax in self.ax]
            #self.C.expose(self.exptime[i]*1.2,frametype='normal',plot=True,ax=self.ax)
            # TO CHANGE EXPOSURE TIME: HERE ####################################################################
            if exptime_test is not None:
                print("Taking test exposure:",exptime_test,"s")
                self.C.expose(exptime_test,frametype='normal',plot=False)
                self.frdimg = FRDImg(self.C.data,self.C.header)
                self.frdimg.plot2_panel(figax=(self.fig,self.ax))
                try:
                    MAX = (np.max(self.frdimg.butter_cut_x)+np.max(self.frdimg.butter_cut_y))/2.
                except Exception as e:
                    print(e)
                    MAX = (np.max(self.frdimg.fimg.data))
                MAXVAL = 47000.
                SCALED_EXPTIME = exptime_test*(MAXVAL/MAX)
                print("Max value was",MAX,"scaling exposure time to ",SCALED_EXPTIME)
                if SCALED_EXPTIME > 20.:
                    SCALED_EXPTIME = 20.
                    print("WARNING: Scaled exptime > 20s. Setting to 20s.")
                self.C.expose(SCALED_EXPTIME,frametype='normal',plot=False)
            else:
                self.C.expose(self.exptime[i]*0.25,frametype='normal',plot=False)
            self.frdimg = FRDImg(self.C.data,self.C.header)
            
            #frdimg.plot2_panel(figax=(fig,ax),plot_folder=PLOT_FOLDER,filename=str(i))
            self.frdimg.plot2_panel(figax=(self.fig,self.ax),plot_folder=self.savefolder,filename=self.filenames[i])
            titlestr = "EXP="+str(self.exptime[i]) + " PERCENT="+str(self.iris_diam[i]) + " STAGE="+str(self.x[i])
            self.fig.suptitle(titlestr)
            self.C.header['IRISDIAM'] = self.iris_diam[i]
            self.C.header['STAGELOC'] = self.x[i]
            self.C.save(self.filenames[i],overwrite=overwrite)
            plt.pause(.3)
            #input_val = raw_input("Press Enter to continue. Press q to stop")
            #if input_val == "q":
            #    break
        
    def close(self):
        self.I.ser.close() # Have to close the serial to the iris (otherwise hangs)

In [4]:
from __future__ import print_function
import FLI
import astropy.io.fits
import astropy
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np

class ControlFLI(object):
    
    def __init__(self,savefolder="."):
        self.C = FLI.USBCamera("flipro0","proline")
        self.header = astropy.io.fits.Header()
        self.header['SERIAL'] = self.C.get_info()['serial_number']
        self.savefolder=savefolder
        
    def expose(self,exptime,frametype="normal",plot=True,ax=None):
        """
        exptime in s
        """
        print("Exposing",exptime,"s...",end="")
        self.header.set("EXPTIME",value=exptime,comment="s")
        self.header['IMAGETYP']= frametype
        self.header.set("DATE-OBS",value=str(datetime.datetime.utcnow()),comment="start of exposure")
        self.C.set_exposure(int(exptime*1000),frametype=frametype)
        self.data = self.C.take_photo()
        self.header.set("CCD-TEMP",value=self.C.get_temperature(),comment="CCD temp in C")
        self.header["DATE-END"] = str(datetime.datetime.utcnow())
        self.header['CAM-MODE'] = self.C.get_camera_mode_string()
        self.header['BASETEMP'] = self.C.read_base_temperature()
        self.header['XBINNING'] = self.C.hbin
        self.header['YBINNING'] = self.C.vbin
        if plot==True:
            if ax == None:
                self.fig, self.ax = plt.subplots()
            else:
                self.ax = ax
            self.ax.imshow(self.data)
            print("MAX COUNTS:",np.max(self.data))
            plt.pause(0.1)
        print('Done')

In [5]:
import os
from astropy.modeling import models, fitting
import photutils
import glob
from astropylib import butter, gklib, gkfit, fitsimg, phothelp, gkastro

class FRDImg(object):
    """
    A class to analyze FRDImages
    """
    
    def __init__(self,data,header):
        self.fimg       = fitsimg.FitsImage(data=data,header=header)
        self.fimg.cropcenter(1024,1056)
        print("Data shape",self.fimg.data.shape)
        
    def plot2_panel(self,fwzm_z=20,figax=None,plot_folder="",filename=""):
        if figax==None:
            fig, ax = plt.subplots(ncols=2,nrows=1,figsize=(10,6))
        else:
            fig, ax = figax
        ax_img         = ax.flat[0]
        ax_cut         = ax.flat[1]
        ax_img.clear()
        ax_cut.clear()
        print("Subtracting,",self.fimg.data.min())
        self.fimg.data -= self.fimg.data.min()
        x,y = self.fimg.get_centroid(plot_lines=True,ax=ax_img)
        self.fimg.ax.set_title("XCEN="+gklib.num2str(x)+" YCEN="+gklib.num2str(y),fontsize=24)
        try:
            print("Cut plot,max=",self.fimg.data.max())
            self.cut_x, self.fwhm_x = self.fimg.get_centroid_line_cut(line="X",return_FWHM=True,plot=True,ax=ax_cut,use_butter_filter=True,butter_cutoff_freq=0.03,fwzm_z=fwzm_z)
            self.cut_y, self.fwhm_y = self.fimg.get_centroid_line_cut(line="Y",return_FWHM=True,plot=True,ax=ax_cut,use_butter_filter=True,butter_cutoff_freq=0.03,fwzm_z=fwzm_z)
            ax_cut.set_ylim(0,64000.)
            self.butter_cut_x = butter.low_pass_butter(self.cut_x,cutoff_freq=0.03)
            self.butter_cut_y = butter.low_pass_butter(self.cut_y,cutoff_freq=0.03)
            ax_cut.plot(self.butter_cut_x)
            ax_cut.plot(self.butter_cut_y)
            _mean = (self.fwhm_x+self.fwhm_y)/2.
            ax_cut.set_title("Radius="+gklib.num2str(_mean/2.),fontsize=24)
        except:
            print("Error!,skipping cut plot")
        if plot_folder!="":
            fig.savefig(plot_folder + filename+".png")
            print("Saved file",plot_folder + filename+".png")
        
    def run(self,
            plot_suffix,
            plot_folder,
            get_rad_at_EE = 0.90,
            MAXRAD_FACTOR = 0.7,
            fwzm_z=20):
        """
        Run the analysis
        
        INPUT:
            plot_suffix = "f01"
            plot_folder = "results/"
            get_rad_at_EE = 0.90
            FACTOR = 0.7
            fwzm_z=20

        OUTPUT:
            Saves files to folders  
        """
        gkastro.make_dir(plot_folder)
        fig, ax = plt.subplots(ncols=3,nrows=2,figsize=(20,15))
        
        ax_img         = ax.flat[0]
        ax_img_bkg     = ax.flat[1]
        ax_img_bkg_sub = ax.flat[2]
        ax_cut         = ax.flat[3]
        ax_ee          = ax.flat[4]
        ax_img_bkg_sub2= ax.flat[5]

        # -------------
        # Plot #1: Normal image
        x,y = self.fimg.get_centroid(plot_lines=True,ax=ax_img)
        self.fimg.ax.set_title("Bias + Dark Subtracted (Hist stretch)")
        #cbar_ax = fig.add_axes([0.38, 0.59, 0.015, 0.3],label="Counts")
        #fig.colorbar(self.fimg.im, cax=cbar_ax)
        
        # -------------
        # Plot #2: Background estimation
        x,y = self.fimg.get_centroid()
        print(x,y)
        #CBS = phothelp.CircularBackgroundSubractor(self.fimg.data,x=x,y=y,r=400)
        #data_back_sub = CBS.subtract_background(plot_background=True,ax=ax_img_bkg)
        #self.subtracted_value = CBS.subtracted_value
        
        # -------------
        # Plot #3: Background subtracted image
        #self.fimg.data = data_back_sub
        #self.fimg.plot(ax=ax_img_bkg_sub,colorbar=False,title="Bias + Dark + Background subtracted (linear stretch)",stretch="linear")
        # add colorbar
        #cbar_ax = fig.add_axes([0.38, 0.09, 0.015, 0.3],label="Counts")
        #fig.colorbar(self.fimg.im, cax=cbar_ax)
        # Get centroid again after taking out the background
        # x,y = self.fimg.get_centroid()        
        
        
        # -------------
        # Plot #4: Cuts
        print("Subtracting:",self.fimg.data.min())
        self.fimg.data -= self.fimg.data.min()
        cut_x, self.fwhm_x = self.fimg.get_centroid_line_cut(line="X",return_FWHM=True,plot=True,ax=ax_cut,use_butter_filter=True,butter_cutoff_freq=0.03,fwzm_z=fwzm_z)
        cut_y, self.fwhm_y = self.fimg.get_centroid_line_cut(line="Y",return_FWHM=True,plot=True,ax=ax_cut,use_butter_filter=True,butter_cutoff_freq=0.03,fwzm_z=fwzm_z)
        
        ax_cut.plot(butter.low_pass_butter(cut_x,cutoff_freq=0.03))
        ax_cut.plot(butter.low_pass_butter(cut_y,cutoff_freq=0.03))
        ax_cut.set_ylim(0,64000.)

        # -------------
        # Plot #4: Encircled energy plot
        
        self.max_radii_for_EE = ((self.fwhm_x + self.fwhm_y)/2.)*MAXRAD_FACTOR
        radii = np.arange(1,self.max_radii_for_EE)
        df, self.r_ee = phothelp.get_encircled_energy_and_rad_at_EE(self.fimg.data,
                                                               self.fimg.xcenter,
                                                               self.fimg.ycenter,
                                                               radii,
                                                               plot=True,
                                                               ax=ax_ee,
                                                               get_rad_at_EE=get_rad_at_EE)
        ax_ee.set_xlim(0,500)
        ax_ee.set_xlabel("Radii (pixels)")
        
        # Overplot aperture in figure $ with EE
        aper = photutils.CircularAperture((x,y),r=self.r_ee)
        aper.plot(ax=ax_img_bkg_sub,color="green")
        ax_img_bkg_sub.annotate("EE"+str(100*get_rad_at_EE)+"%",xy=(x,y+self.r_ee),color="white",fontsize=8)
        # overplot max aperture 
        aper = photutils.CircularAperture((x,y),r=self.max_radii_for_EE)
        aper.plot(ax=ax_img_bkg_sub,color="red")
        ax_img_bkg_sub.annotate("EE100% @ r = "+str(MAXRAD_FACTOR*2)+" HWHM",xy=(x,y+self.max_radii_for_EE),color="white",fontsize=8)
        
        # Hist plot
        self.fimg.plot(ax=ax_img_bkg_sub2,colorbar=False,title="Bias + Dark + Background subtracted (Hist stretch)",stretch="hist")
        aper = photutils.CircularAperture((x,y),r=self.r_ee)
        aper.plot(ax=ax_img_bkg_sub2,color="green")
        ax_img_bkg_sub2.annotate("EE"+str(100*get_rad_at_EE)+"%",xy=(x,y+self.r_ee),color="white",fontsize=8)
        # overplot max aperture 
        aper = photutils.CircularAperture((x,y),r=self.max_radii_for_EE)
        aper.plot(ax=ax_img_bkg_sub2,color="red")
        ax_img_bkg_sub2.annotate("EE100% @ r = "+str(MAXRAD_FACTOR*2)+" HWHM",xy=(x,y+self.max_radii_for_EE),color="white",fontsize=8)

        fig.tight_layout()
        fig.subplots_adjust(top=0.9)
        fig.suptitle("FRD Overview Plot, file: ",fontsize=20)
        #fig.savefig(plot_folder + self.basename+"_"+plot_suffix+".png")
        #print("Saved file:",plot_folder + self.basename+"_"+plot_suffix+".png")
        #df.to_csv(plot_folder + self.basename+"_"+plot_suffix+".csv")
        #print("Saved file:",plot_folder + self.basename+"_"+plot_suffix+".csv")
        
    
def get_EE_from_rad_in_pix(df,radius_in_pix):
    return df[df["radii"] > radius_in_pix]["EE"].values[0]

def add_y_in_input(df_config):
    df_config["y_in_input"] = (df_config["f_ratio_out"]/df_config["f_ratio_in"])*df_config["y_out_fiber_dist"]
    return df_config

def resample_df_mean(df):
    chunks = gkastro.chunker(df,4)
    dff = pd.DataFrame()
    for chunk in chunks:
        dff = pd.concat([dff,pd.DataFrame(chunk[["f_ratio_in","f_ratio_out","EE_in_input_cone"]].median(skipna=True)).T],ignore_index=True)
    return dff


# ^^ Run everything above this ^^

# Main thing to run

In [17]:
#FRD = FRDSequencer(savefolder='C:\\Users\\PSM-0\\Desktop\\HPF_FRD\\20170801_test_puck_autocollimated_nohene_50micron\\f02')
try:
    FRD.close()
except Exception as e:
    print('No FRD instance, no need to close.')
FRD = FRDSequencer(csv_setup_file="setup_f01.csv",
                       savefolder='C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/')

Connecting to devices...Serial is 83829712
initializeHardwareDevice serial c_long(83829712)
initializeHardwareDevice connection SUCESS
Stage...Camera...Stage...Connection established
Done
Reading sequence file...Done


In [7]:
FRD.init()

Going home
go_home SUCESS
Initialized


In [18]:
FRD.S.go_to_32()

mAbs  32.0 c_float(32.0)
mAbs SUCESS


In [19]:
FRD.S.go_to(29)

mAbs  29 c_float(29.0)
mAbs SUCESS


In [15]:
FRD.I.set_percent(2)

In [16]:
FRD.I.set_percent(18)

In [ ]:
FRD.close()

In [ ]:
FRD.I.set_percent(11)

In [20]:
####### 
# Run full sequence

# 1. Is PSF centered?
# 2. Is PSF as close to shutter as possible at 32?
# 3. Is it set to save in the right directory?
# 4. Is light contamination minimized ? Screen off, etc. 
# 5. Is the box on?


FRD.run_sequence(overwrite=False,exptime_test=0.2)

##########################################
##########################################
Running sequence
Savefolder: C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/
##########################################
Setting the percent as 18 in the beginning
##########################################
PERCENT=: 18
mAbs  32.0 c_float(32.0)
mAbs SUCESS
0 STAGE AT= 32.0 mm
Taking test exposure: 0.2 s
Exposing 0.2 s...Done
Data shape (1027L, 1024L)
Subtracting, 2159.0
Cut plot,max= 16369.0
Max value was 13845.9021173 scaling exposure time to  0.678901231599
Exposing 0.678901231599 s...Done
Data shape (1027L, 1024L)
Subtracting, 2208.0
Cut plot,max= 47664.0
Saved file C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/f_psm_01_d_01.fits.png
Saved to C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/\f_psm_01_d_01.fits
##########################################
PERCENT=: 18
mAbs  31.0 c_float(31.0)
mAbs SUCESS
1 STAGE AT= 31.0 mm

##########################################
PERCENT=: 12
mAbs  31.0 c_float(31.0)
mAbs SUCESS
13 STAGE AT= 31.0 mm
Taking test exposure: 0.2 s
Exposing 0.2 s...Done
Data shape (1027L, 1024L)
Subtracting, 2161.0
Cut plot,max= 14027.0
Max value was 12283.7838541 scaling exposure time to  0.765236519272
Exposing 0.765236519272 s...Done
Data shape (1027L, 1024L)
Subtracting, 2210.0
Cut plot,max= 45895.0
Saved file C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/f_psm_04_d_02.fits.png
Saved to C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/\f_psm_04_d_02.fits
##########################################
PERCENT=: 12
mAbs  30.0 c_float(30.0)
mAbs SUCESS
14 STAGE AT= 30.0 mm
Taking test exposure: 0.2 s
Exposing 0.2 s...Done
Data shape (1027L, 1024L)
Subtracting, 2151.0
Cut plot,max= 12403.0
Max value was 11094.4846201 scaling exposure time to  0.847267838197
Exposing 0.847267838197 s...Done
Data shape (1027L, 1024L)
Subtracting, 2219.0
Cut plo

Exposing 0.913448415428 s...Done
Data shape (1027L, 1024L)
Subtracting, 2222.0
Cut plot,max= 43803.0
Saved file C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/f_psm_07_d_03.fits.png
Saved to C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/\f_psm_07_d_03.fits
##########################################
PERCENT=: 9
mAbs  29.0 c_float(29.0)
mAbs SUCESS
27 STAGE AT= 29.0 mm
Taking test exposure: 0.2 s
Exposing 0.2 s...Done
Data shape (1027L, 1024L)
Subtracting, 2159.0
Cut plot,max= 10817.0
Max value was 9469.20270861 scaling exposure time to  0.992691812527
Exposing 0.992691812527 s...Done
Data shape (1027L, 1024L)
Subtracting, 2220.0
Cut plot,max= 44991.0
Saved file C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/f_psm_07_d_04.fits.png
Saved to C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/\f_psm_07_d_04.fits
##########################################
PERCENT=: 7
mAbs  32.0 c_float(32

##########################################
PERCENT=: 3
mAbs  32.0 c_float(32.0)
mAbs SUCESS
40 STAGE AT= 32.0 mm
Taking test exposure: 0.2 s
Exposing 0.2 s...Done
Data shape (1027L, 1024L)
Subtracting, 2139.0
Cut plot,max= 5927.0
Max value was 5338.65298104 scaling exposure time to  1.76074377439
Exposing 1.76074377439 s...Done
Data shape (1027L, 1024L)
Subtracting, 2264.0
Cut plot,max= 41842.0
Saved file C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/f_psm_11_d_01.fits.png
Saved to C://Users//PSM-0//Desktop//NEID_FRD/20180607_science6_brass_both_50um/HE2/\f_psm_11_d_01.fits
##########################################
PERCENT=: 3
mAbs  31.0 c_float(31.0)
mAbs SUCESS
41 STAGE AT= 31.0 mm
Taking test exposure: 0.2 s
Exposing 0.2 s...Done
Data shape (1027L, 1024L)
Subtracting, 2142.0
Cut plot,max= 5346.0
Max value was 4834.73955022 scaling exposure time to  1.94426191988
Exposing 1.94426191988 s...Done
Data shape (1027L, 1024L)
Subtracting, 2274.0
Cut plot,max= 4

# Extra testing

In [ ]:
#PLOT_FOLDER="C://Users//PSM-0//Desktop//HPF_FRD/20170901_test_puck_D_200micron_hene/injection_images/"
#fig, ax = plt.subplots(ncols=2,nrows=1,figsize=(18,10))
#EXPTIME = 0.6
#for i in range(10):
#   C = ControlFLI()
#    C.expose(EXPTIME,plot=False)
#    frdimg = FRDImg(C.data,C.header)
#    #frdimg.plot2_panel(figax=(fig,ax),plot_folder=PLOT_FOLDER,filename=str(i))
#   frdimg.plot2_panel(figax=(fig,ax),plot_folder=PLOT_FOLDER,filename="exp"+str(i)+"exp"+str(EXPTIME))
#    plt.pause(1)
#
#C = ControlFLI()
#C.expose(0.6,plot=False)
#frdimg = FRDImg(C.data,C.header)
#frdimg.run(get_rad_at_EE=0.96,MAXRAD_FACTOR=0.54,fwzm_z=20.,plot_suffix="f01",plot_folder=PLOT_FOLDER)
#fimg = astropylib.fitsimg.FitsImage(data=C.data,header=C.header)
#fimg.plot()